In [1]:
import ee
import math
import urllib
import subprocess
import os 
import glob
import geopandas as gpd
import rasterio 
from rasterio import features
from shapely.geometry import Polygon
from scipy.misc import imsave, imshow
import matplotlib.pyplot as plt
import numpy as np 
from matplotlib import cm 

import ipywidgets as widgets
%matplotlib inline

ee.Initialize()

In [2]:
city2latlon = {
    'austin': (30.2672, -97.7431),
    'dc': (38.9072, -77.0369),
    'denver': (39.7392, -104.9903),
    'medellin': (6.2442, -75.5812),
    'madrid': (40.4168, -3.7038),
    'budapest': (47.4979, 19.0402),
    'berlin': (52.5200, 13.4050),
    'athina': (37.9838, 23.7275),
    'barcelona': (41.3851, 2.1734),
    'roma': (41.9028, 12.4964),
    'hamburg': (53.5511, 9.9937),
    'nantes': (47.2184, -1.5536),
    'dublin': (53.3498, -6.2603),
    'amsterdam': (52.3702, 4.8952),
    'malmo': (55.6050, 13.0038),
    'helsinki': (60.1699, 24.9384),
    'birmingham': (52.4862, -1.8904),
    'belfast': (54.5973, -5.9301),
    'lisboa': (38.7223, -9.1393),
    'bologna': (44.4949, 11.3426),
    'bruxelles': (50.8503, 4.3517),
    'wien': (48.2082, 16.3738),
    'graz': (47.0707, 15.4395), 
    'innsbruck': (47.2692, 11.4041), 
    'varna': (43.2141, 27.9147), 
    'ruse': (43.8356, 25.9657), 
    'brno': (49.1951, 16.6068), 
    'ostrava': (49.8209, 18.2625), 
    'hannover': (52.3759, 9.7320), 
    'cordoba': (37.8882, -4.7794), 
    'rennes': (48.1173, -1.6778), 
    'napoli': (40.8518, 14.2681), 
    'genova': (44.4056, 8.9463), 
    'riga': (56.9496, 24.1052), 
    'valletta': (35.8989, 14.5146), 
    'rotterdam': (51.9244, 4.4777),
    'salzburg':(47.8095, 13.0550),
     'antwerpen':(51.2194, 4.4025),
     'gent':(51.0543, 3.7174),
     'plzen':(49.7384, 13.3736),
     'munchen':(48.1351, 11.5820),
     'main':(50.1109, 8.6821),
     'bremen':(53.0793, 8.8017),
     'bielefeld':(52.0302, 8.5325),
     'essen':(51.4556, 7.0116),
     'aalborg':(57.0488, 9.9217),
     'palmas':(28.1235, -15.4363),
     'vigo':(42.2406, -8.7207),
     'paris':(48.8566, 2.3522),
     'lyon':(45.7640, 4.8357),
     'toulouse':(43.6047, 1.4442),
     'marseille':(43.2965, 5.3698),
     'torino':(45.0703, 7.6869),
     'verona':(45.4384, 10.9916),
     'luxembourg':(49.8153, 6.1296),
     'warszawa':(52.2297, 21.0122),
     'katowice':(50.2649, 19.0238),
     'stockholm':(59.3293, 18.0686),
     'goteborg':(57.7089, 11.9746),
     'bruggie': (51.2093, 3.2247),
     'plovdiv': (42.1354, 24.7453),
     'leipzig': (51.3397, 12.3731),
     'dusseldorf': (51.2277, 6.7735),
     'gottingen': (51.5413, 9.9158),
     'koblenz': (50.3569, 7.5890),
     'uppsala':(59.8586, 17.6389),
     'linkoping':(58.4108, 15.6214),
     'orebro':(59.2753, 15.2134),
     'ljubljana':(46.0569, 14.5058),
     'manchester':(53.4808, -2.2426),
     'leeds':(53.801277, -1.548567),
    'linz': (48.3069, 14.2858),
    'gent': (51.0543, 3.7174),
    'charleroi': (50.4108, 4.4446),
    'liege': (50.6326, 5.5797),
    'sofia': (42.6977, 23.3219),
    'burgas': (42.5048, 27.4626),
    'pleven': (43.4170, 24.6067),
    'vidin': (43.9962, 22.8679),
    'starazagora': (42.4258, 25.6345),
    'lefkosia': (35.1856, 33.3823),
    'praha': (50.0755, 14.4378),
    'koln': (50.9375, 6.9603),
    'stuttgart': (48.7758, 9.1829),
    'dresden': (51.0504, 13.7373),
    'nurnberg': (49.4521, 11.0767),
    'trier': (49.7500, 6.6371),
    'augsburg': (48.3705, 10.8978),
    'bonn': (50.7374, 7.0982),
    'karlsruhe': (49.0069, 8.4037),
    'kobenhavn': (55.6761, 12.5683),
    'aarhus': (56.1629, 10.2039),
    'odense': (55.4038, 10.4024),
    'metz': (49.1193, 6.1757),
    'ferrand': (45.7772, 3.0870),
    'grenoble': (45.1885, 5.7245),
    'nice': (43.7102, 7.2620),
    'potenza': (40.6464, 15.8055),
    'katowice': (50.2649, 19.0238),
    'oporto': (41.1579, -8.6291),
    'linkoping': (58.4108, 15.6214),
    'orebro': (59.2753, 15.2134),
    'tyne': (54.9783, -1.6178),
    'abredeen': (57.1497, -2.0943),
    'bogota': (4.7110, -74.0721),
    'nairobi': (-1.2921, 36.8219)
}

def bounding_box_at_location(latlon, sizeKm):
    widthKm, heightKm = sizeKm
    latitudeInDegrees, longitudeInDegrees = latlon
    # degrees to radians
    def deg2rad(degrees):
        return math.pi*degrees/180.0

    # radians to degrees
    def rad2deg(radians):
        return 180.0*radians/math.pi

    # Semi-axes of WGS-84 geoidal reference
    WGS84_a = 6378137.0  # Major semiaxis [m]
    WGS84_b = 6356752.3  # Minor semiaxis [m]

    # Earth radius at a given latitude, according to the WGS-84 ellipsoid [m]
    def WGS84EarthRadius(lat):
        # http://en.wikipedia.org/wiki/Earth_radius
        An = WGS84_a*WGS84_a * math.cos(lat)
        Bn = WGS84_b*WGS84_b * math.sin(lat)
        Ad = WGS84_a * math.cos(lat)
        Bd = WGS84_b * math.sin(lat)
        return math.sqrt( (An*An + Bn*Bn)/(Ad*Ad + Bd*Bd) )

    lat = deg2rad(latitudeInDegrees)
    lon = deg2rad(longitudeInDegrees)
    widthMeters = 1000*widthKm/2.0
    heightMeters = 1000*heightKm/2.0

    # Radius of Earth at given latitude
    radius = WGS84EarthRadius(lat)
    # Radius of the parallel at given latitude
    pradius = radius*math.cos(lat)

    latMin = lat - heightMeters/radius
    latMax = lat + heightMeters/radius
    lonMin = lon - widthMeters/pradius
    lonMax = lon + widthMeters/pradius

    return (rad2deg(latMin), rad2deg(lonMin), rad2deg(latMax), rad2deg(lonMax))

In [12]:
bounding_box_at_location((-23.5505, -46.6333), (40,40))

(-23.730258598084127,
 -46.82939139034745,
 -23.37074140191587,
 -46.43720860965254)

# Google Earth Engine Download

In [9]:
def download_landsat(latmin, lonmin, latmax, lonmax, city) :
    
    coords = [[lonmin, latmin], [lonmax, latmin], [lonmax, latmax], [lonmin, latmax], [lonmin, latmin]]
    city_p = ee.Geometry.Polygon(coords)

    start = '2016-07-25'
    finish = '2017-07-25'

    cc_per = 0.1
    while True :
        try :
            image_collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterBounds(city_p)\
                .filterDate(start, finish).filter(ee.Filter(ee.Filter.lt('CLOUD_COVER', cc_per)));

            reduced = ee.Image(image_collection.median()).clip(city_p);

            # pan-sharpen
            hsv = reduced.select(['B4', 'B3', 'B2']).rgbToHsv();
            sharpened = ee.Image.cat([
              hsv.select('hue'), hsv.select('saturation'), reduced.select('B8')
            ]).hsvToRgb();

            rgb = sharpened.visualize(['red', 'green', 'blue'], None, None, 0, 0.25, [1.3, 1.3, 1.3])
            renamed = rgb.select(['vis-red', 'vis-green', 'vis-blue'], ['r', 'g', 'b'])
            url = renamed.getDownloadURL({'scale': 13, 'crs': 'EPSG:4326'})
            break
        except Exception :
            cc_per += 0.05
            if cc_per == 0.2 :
                print('{} requires more than 20% cloud cover...'.format(city))
                raise

    if os.path.exists('landsat/{}'.format(city)) :
        print('landsat already downloaded...')
        return 
    
    urllib.urlretrieve(url, 'landsat/{}.zip'.format(city))

    if not os.path.exists('landsat/{}'.format(city)) :
        os.mkdir('landsat/{}'.format(city))
    else :
        subprocess.call('rm landsat/{}/*'.format(city), shell=True)

    subprocess.call('tar xvf landsat/{0}.zip -C landsat/{0}'.format(city), shell=True)

# Convert Shapefile to Raster based on city limits

In [10]:
original_classes = '''Agricultural + Semi-natural areas + Wetlands
Airports
Construction sites
Continuous Urban Fabric (S.L. > 80%)
Discontinuous Dense Urban Fabric (S.L. : 50% -  80%)
Discontinuous Low Density Urban Fabric (S.L. : 10% - 30%)
Discontinuous Medium Density Urban Fabric (S.L. : 30% - 50%)
Discontinuous Very Low Density Urban Fabric (S.L. < 10%)
Fast transit roads and associated land
Forests
Green urban areas
Industrial, commercial, public, military and private units
Isolated Structures
Land without current use
Mineral extraction and dump sites
Other roads and associated land
Port areas
Railways and associated land
Sports and leisure facilities
Water bodies'''.split("\n")

def filter_(df, sp, p) :
    possible_matches_index = list(sp.intersection(p.bounds))
    possible_matches = df.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(p)]
    return precise_matches

consolidate_classes = {
    "Continuous Urban Fabric (S.L. > 80%)":"High Density Urban Fabric",
     "Discontinuous Dense Urban Fabric (S.L. : 50% -  80%)":"High Density Urban Fabric",
     "Discontinuous Medium Density Urban Fabric (S.L. : 30% - 50%)":"Discontinuous Urban Fabric",
     "Discontinuous Low Density Urban Fabric (S.L. : 10% - 30%)":"Discontinuous Urban Fabric",
     "Discontinuous Very Low Density Urban Fabric (S.L. < 10%)":"Discontinuous Urban Fabric",
    "Airports":"Transportation",
    "Fast transit roads and associated land":"Transportation",
    "Railways and associated land": "Transportation",
    "Port areas":"Transportation",
    "Other roads and associated land": "Transportation",
    "Mineral extraction and dump sites": "Other land",
    "Land without current use": "Other land",
    "Construction sites": "Other land",
    "Isolated Structures": "Other land",
    'Sports and leisure facilities': "Green urban areas",
    "Industrial, commercial, public, military and private units": "Discontinuous Urban Fabric"
}

new_classes = ['Transportation', 
               'High Density Urban Fabric', 
               'Discontinuous Urban Fabric',
       u'Agricultural + Semi-natural areas + Wetlands', 
               'Other land',
       u'Green urban areas', 
               u'Forests', 
               u'Water bodies']

class2label = {c:i for i,c in enumerate(new_classes)}

In [12]:
# unzip urban atlas only once and rename to city

def create_raster(city) :
    if not os.path.exists('urban_atlas/{}'.format(city)) :
        filename = os.path.basename([f for f in glob.glob('urban_atlas/*.zip') if city in f][0]).split('.')[0]
        subprocess.call('tar xvf urban_atlas/{}.zip -C urban_atlas/ && mv urban_atlas/{} urban_atlas/{}' \
                        .format(filename, filename, city), shell=True)

    # get transform from tif file 
    tif = rasterio.open(glob.glob('landsat/{}/*.tif'.format(city))[0])
    transform = tif.transform
    shape = tif.shape
    left, bottom, right, top = list(tif.bounds)
    tif.close()

    # open shapefile and change coordinate system 
    shapefile = glob.glob('urban_atlas/{}/*.shp'.format(city))[0]
    df = gpd.GeoDataFrame.from_file(shapefile)

    targetcrs = {u'ellps': u'WGS84', u'datum': u'WGS84', u'proj': u'longlat'}
    df.to_crs(crs=targetcrs, inplace=True)

    df['ITEM'] = df['ITEM'].apply(
        lambda x: consolidate_classes[x] if x in consolidate_classes else x)

    # filter shapefile by landsat image bounds 
    city_bounds = Polygon([[left, bottom], [left, top], [right, top], [right, bottom], [left, bottom]])
    sindex = df.sindex
    matches = filter_(df, sindex, city_bounds)

    matches['CLASS'] = matches['ITEM'].apply(lambda x : class2label[x])
    
    assert(len(matches['CLASS'].unique()) <= len(new_classes))
    
    # create raster 
    shapes = zip(matches['geometry'].values, matches['CLASS'].values)
    raster = features.rasterize(shapes, out_shape=(shape[0], shape[1]), transform=transform, all_touched=True, fill=len(new_classes))
    return raster

# Grid landsat and raster into training examples

In [12]:
# 8 total classes, 9 counting broken class 

def save_grids(city, raster=None) :

    raw_tif = glob.glob('landsat/{}/*.tif'.format(city))
    r = rasterio.open([path for path in raw_tif if '.r.' in path][0]).read()
    g = rasterio.open([path for path in raw_tif if '.g.' in path][0]).read()
    b = rasterio.open([path for path in raw_tif if '.b.' in path][0]).read()
    satellite = np.stack([r, g, b], axis=0).squeeze(1).transpose([1, 2, 0])

    height, width = satellite.shape[:2]
    r, c = int(height/256.), int(width/256.)
    
    skipped = 0 
    found = 0
    new_skipped = 0
    broken = 0
    blacky = 0
    for r_ix in range(r) :
        for c_ix in range(c) :
            x = satellite[r_ix*256:(r_ix+1)*256, c_ix*256:(c_ix+1)*256]
            
            if not (raster is None) :
                y = raster[r_ix*256:(r_ix+1)*256, c_ix*256:(c_ix+1)*256]
                if len(np.unique(y)) == 1 :
                    skipped += 1
                    continue
                
                unique, counts = np.unique(y, return_counts=True)
                counts = counts*1.0 / y.size
                if np.max(counts) > 0.9: 
                    new_skipped += 1
                    continue 
                if 8 in list(unique) :
                    d = dict(zip(list(unique), list(counts)))
                    if d[8] > 0.1 :
                        broken += 1
                        continue 

            if len(np.unique(x)) == 1:
                skipped += 1
                continue 
                if 8 in list(unique) :
                    d = dict(zip(list(unique), list(counts)))
                    if d[8] > 0.1 :
                        broken += 1
                        continue 
            
            num_black = 0
            for j in range(256) :
                for k in range(256) :
                    if list(x[j, k, :]) == [0, 0, 0] :
                        num_black = num_black + 1
            if num_black/(256*256.) > 0.2 :
                blacky += 1
                continue
            
            found += 1
            np.save('dataset/labels/{}_{}_{}'.format(city, r_ix, c_ix), y)
            imsave('dataset/satellite/{}_{}_{}.jpg'.format(city, r_ix, c_ix), x)
            
    print('{} found, {} skipped {} new skipped, {} broken {} too black...'.format(found, skipped, new_skipped, broken, blacky))

# Run for all train cities

In [14]:
import traceback
import logging

cities = city2latlon.keys()
window = (40, 40) # km

for i, city in enumerate(cities) :
    print('{} {}/{}'.format(city, i, len(cities)))
    
    
    try : 
        latmin, lonmin, latmax, lonmax = bounding_box_at_location(city2lonlat[city], window)
    except :
        print('problem with bounding box calculation...')
        continue
    
    try :
        download_landsat(latmin, lonmin, latmax, lonmax, city)
    except Exception as e :
        continue
    
    try :
        raster = create_raster(city)
    except Exception as e:
        print('problem with raster creation...')
        traceback.print_exc()
        continue
    
    try :
        save_grids(city, raster)
    except Exception as e:
        print('problem with saving grids...')
        continue

plzen 0/96
landsat already downloaded...
139 found, 0 skipped 5 new skipped, 0 broken 0 blacky...
sofia 1/96
sofia requires more than 20% cloud cover...
birmingham 2/96
birmingham requires more than 20% cloud cover...
belfast 3/96
belfast requires more than 20% cloud cover...
ferrand 4/96
ferrand requires more than 20% cloud cover...
oporto 5/96
landsat already downloaded...
49 found, 63 skipped 7 new skipped, 25 broken 0 blacky...
paris 6/96
landsat already downloaded...
143 found, 0 skipped 1 new skipped, 0 broken 0 blacky...
nurnberg 7/96
landsat already downloaded...
81 found, 5 skipped 17 new skipped, 41 broken 0 blacky...
ljubljana 8/96
ljubljana requires more than 20% cloud cover...
rotterdam 9/96
rotterdam requires more than 20% cloud cover...
innsbruck 10/96
innsbruck requires more than 20% cloud cover...
budapest 11/96
landsat already downloaded...
73 found, 54 skipped 2 new skipped, 3 broken 12 blacky...
malmo 12/96
malmo requires more than 20% cloud cover...
pleven 13/96
pl

# Download test cities

In [8]:
import subprocess 

def save_grids(city) :

    raw_tif = glob.glob('test_landsat/{}/*.tif'.format(city))
    r = rasterio.open([path for path in raw_tif if '.r.' in path][0]).read()
    g = rasterio.open([path for path in raw_tif if '.g.' in path][0]).read()
    b = rasterio.open([path for path in raw_tif if '.b.' in path][0]).read()
    satellite = np.stack([r, g, b], axis=0).squeeze(1).transpose([1, 2, 0])

    height, width = satellite.shape[:2]
    r, c = int(height/224.), int(width/224.)

    skipped = 0 
    found = 0
    for r_ix in range(r) :
        for c_ix in range(c) :
            x = satellite[r_ix*224:(r_ix+1)*224, c_ix*224:(c_ix+1)*224]

            if len(np.unique(x)) == 1:
                skipped += 1
                continue 

            found += 1
            imsave('test_grids/{0}/{0}_{1}_{2}.jpg'.format(city, r_ix, c_ix), x)

    print('{} found, {} skipped...'.format(found, skipped))

cities = ['austin']
years = ['2015']
window = (40, 40) # km

for i, city in enumerate(cities) :
    for year in years :
        
        filename = str(city)+str(year)

        if not os.path.exists('test_landsat/{}'.format(str(city)+str(year))) :
            os.mkdir('test_landsat/{}'.format(filename))
            subprocess.call('tar xvf test_landsat/{0}.zip -C test_landsat/{0}'.format(filename), shell=True)
        
        
        save_grids(filename)

195 found, 0 skipped...
